## chartEntry Datastructure

In [1]:
from __future__ import print_function
import pandas as pd
from pandas.io.parsers import read_csv
import heapq as heapq
import numpy as np

class chartEntry:
    """
    This class implements the data structure "Entry" described in the baseline
    algorithm. The pseduocode for the description is:
    
    Entry(word, start-position, end-position, log-probability, back-pointer)
    
    We define the __lt__ and __eq__ operators to allow comparisons betwen two entries when 
    trying to push them into the heap. This operator respects the sort_acc_to variable and 
    provides boolean value accordingly. 

    It supports operations based on two ideas - i) sorting by start_pos
                                                ii) sorting by log_prob. 
        
    IMPORTANT NOTE: For Comparison, two objects can be compared based upon their start-pos 
    or log-prob (depending upon the initial setting of sort_acc_to when defining the objects) 
    using the normal comparison operators like < or >. HOWEVER, equals comparison SHOULD ONLY 
    BE MADE USING isEquals() function, and NOT ==. 
    
    
    EXAMPLE USAGE: 
        p = chartEntry('Anmol', 0, 4, 0.2, -1, sort_acc_to='start_pos')
        print(p)
        p.get_item('log_prob')
        e1 = chartEntry('Anmol', 0, 4, 0.2, -1)
        e2 = chartEntry("Shreeashish", 5, 10, 0.5, 0)
        e3 = chartEntry('Amir Ali', 11, 16, 0.4, 1)
        p1 = chartEntry('Anmol', 0, 4, 0.2, -1, sort_acc_to='start_pos')
        p2 = chartEntry('Anmol', 0, 4, 0.2, -1, sort_acc_to='start_pos')
        p1.isEqual(p2) [TRUE]
        
    This work is a part of the Assignment 1 of CMPT 825 Natural Language Processing
    taught by Prof. Anoop Sarkar. 
    
    AUTHOR: Anmol Sharma, GroupNLP
    INSTITUTION: Simon Fraser University
    """
    
    def __init__(self, word, start_pos, end_pos, log_prob, back_ptr, sort_acc_to='start_pos'):
        self.instance = {}
        self.instance['word'] = word
        self.instance['start_pos'] = start_pos
        self.instance['end_pos'] = end_pos
        self.instance['log_prob'] = log_prob
        self.instance['back_ptr'] = back_ptr
        self.__sort_type = sort_acc_to
        
    def __repr__(self):
        return "chartEntry({}, {}, {}, {}, {})".format(self.instance['word'], self.instance['start_pos'], \
                                              self.instance['end_pos'], self.instance['log_prob'],\
                                              self.instance['back_ptr'])
    
    def __lt__(self, other_obj):
        if self.__sort_type == 'start_pos':
            return (self.instance['start_pos'] < other_obj.instance['start_pos'])
        else:
            return (self.instance['log_prob'] < other_obj.instance['log_prob'])
    
    def __gt__(self, other_obj):
        if self.__sort_type == 'start_pos':
            return (self.instance['start_pos'] > other_obj.instance['start_pos'])
        else:
            return (self.instance['log_prob'] > other_obj.instance['log_prob'])
    
    def __eq__(self, other_obj):
        if self.__sort_type == 'start_pos':
            return (self.instance['start_pos'] == other_obj.instance['start_pos'])
        else:
            return (self.instance['log_prob'] == other_obj.instance['log_prob'])
    
    def isEqual(self, other_obj):
        for k in self.instance:
            if self.instance[k] == other_obj.instance[k]:
                continue
            else:
                return False
        return True
    
    def get_item(self, key):
        return self.instance[key] if key in self.instance else "Undefined Key"
    

## Start building the heap

In [2]:
class Heap:
    """
    A class wrapper for heapq datastructure implementation of python. Python's default heapq 
    implementation requires a list as initialized heap, however it doesn't provide
    any safeguards against the fact that the underlying list may be changed by some function. 
    
    To provide safeguard mechanism, this class wraps the push and pop functions of heapq. 
    
    EXAMPLE USAGE:
        sort_acc_to='log_prob'
        p = chartEntry('Anmol', 0, 4, 0.2, -1, sort_acc_to)
        print(p)
        p.get_item('log_prob')
        e1 = chartEntry('Anmol', 0, 4, 0.6, -1, sort_acc_to)
        e2 = chartEntry("Shreeashish", 5, 10, 0.5, 0, sort_acc_to)
        e3 = chartEntry('Amir Ali', 11, 16, 0.1, 1, sort_acc_to)
        heap1 = Heap()
        heap1.push(e1)
        heap1.push(e2)
        heap1.push(e3)
        heap1.pop()
        
    This work is a part of the Assignment 1 of CMPT 825 Natural Language Processing
    taught by Prof. Anoop Sarkar. 
    
    AUTHOR: Anmol Sharma, GroupNLP
    INSTITUTION: Simon Fraser University
    """
    
    def __init__(self, ls=None):
        self.__heap = ls if ls else []
        heapq.heapify(self.__heap)
        
    def push(self, item):
        heapq.heappush(self.__heap, item)
    
    def pop(self):
        return heapq.heappop(self.__heap)
    
    def __len__(self):
        return len(self.__heap)
    
    def __repr__(self):
        return "{}".format(self.__heap)

## Let's start implemeting the algorithm

In [3]:
class Pdist(dict):
    "A probability distribution estimated from counts in datafile."

    def __init__(self, filename, sep='\t', N=None, missingfn=None):
        self.maxlen = 0 
        for line in file(filename):
            (key, freq) = line.split(sep)
            try:
                utf8key = unicode(key, 'utf-8')
            except:
                raise ValueError("Unexpected error %s" % (sys.exc_info()[0]))
            self[utf8key] = self.get(utf8key, 0) + int(freq)
            self.maxlen = max(len(utf8key), self.maxlen)
        self.N = float(N or sum(self.itervalues()))
        self.missingfn = missingfn or (lambda k, N: 1./N)

    def __call__(self, key):
        if key in self: return float(self[key])/float(self.N)
        #else: return self.missingfn(key, self.N)
        elif len(key) == 1: return self.missingfn(key, self.N)
        else: return None

In [5]:
# build the probability distribution over the unigram frequencies in count_1w
Pw  = Pdist('../data/count_1w.txt')

## Heap initialization

In [13]:
"""
NOTE: Correct conversion of str to unicode is highly important in this example, so need to be
careful whenever we use anything related to the words themselves. Also, remember when you "print"
a unicode character, it will print the actual character (the chinese letter), but if you view
it in a list, it will show up as a unicode string. 
"""
def initHeap(input_filename='../data/input', counts_filename='../data/count_1w.txt', \
             sort_acc_to='log_prob'):
    
    # initialize heap and open input and counts file
    heap = Heap()
    input_file = open(input_filename, 'rb')
    counts_file = open(counts_filename, 'rb')

    # read the first line from the input file and convert it into a list of characters
    input_line = input_file.readline()
    input_chars = [i for i in unicode(input_line, 'utf-8')]

    # iterate through the counts file to find all instances which match the first character at position 0
    for word_line in counts_file:
        key, freq = word_line.split('\t')
        word_chars = [i for i in unicode(key, 'utf-8')]

        
        # compare the first character of the word from counts file to the first character of input line
         if word_chars[0] == input_chars[0]:
            print('Input: {}'.format("".join(input_chars).encode('utf-8')))
            print('Matched: {}\n'.format("".join(word_chars).encode('utf-8')))

            # create a new entry. Note the .encode('utf-8) which is very important here. We use log_2 
            # probability here. 
            entry = chartEntry("".join(word_chars).encode('utf-8'), start_pos=0, end_pos=len(word_chars),\
                                                log_prob=np.log2(Pw("".join(word_chars))), back_ptr=None,\
                                                sort_acc_to=sort_acc_to)
            print(entry, '\n')

            # push the new entry to the heap
            heap.push(entry)
            
            
    return heap

In [14]:
heap = initHeap(input_filename='../data/input', counts_filename='../data/count_1w.txt', \
             sort_acc_to='log_prob')

Input: 法正研究从波黑撤军计划

Matched: 县长

chartEntry(县长, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 微型

chartEntry(微型, 0, 2, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 漳州

chartEntry(漳州, 0, 2, -14.0545195309, None) 

Input: 法正研究从波黑撤军计划

Matched: 游乐园

chartEntry(游乐园, 0, 3, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 谈

chartEntry(谈, 0, 1, -14.0545195309, None) 

Input: 法正研究从波黑撤军计划

Matched: 第二

chartEntry(第二, 0, 2, -11.9170160071, None) 

Input: 法正研究从波黑撤军计划

Matched: 还要

chartEntry(还要, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 副市长

chartEntry(副市长, 0, 3, -14.791485125, None) 

Input: 法正研究从波黑撤军计划

Matched: 第五

chartEntry(第五, 0, 2, -14.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 居于

chartEntry(居于, 0, 2, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 为主

chartEntry(为主, 0, 2, -12.5690927037, None) 

Input: 法正研究从波黑撤军计划

Matched: 吸纳

chartEntry(吸纳, 0, 2, -14.0545195309, None) 

Input: 法正研究从波黑撤军计划

Matched: 韩正

chartEntry(韩正, 0, 2, -16.37644


chartEntry(天, 0, 1, -11.4222513154, None) 

Input: 法正研究从波黑撤军计划

Matched: 级别

chartEntry(级别, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 谭刚伦

chartEntry(谭刚伦, 0, 3, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 比什凯克

chartEntry(比什凯克, 0, 4, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 纺织品

chartEntry(纺织品, 0, 3, -13.791485125, None) 

Input: 法正研究从波黑撤军计划

Matched: 地质

chartEntry(地质, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 侵占

chartEntry(侵占, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 采用

chartEntry(采用, 0, 2, -12.6760079076, None) 

Input: 法正研究从波黑撤军计划

Matched: 百分之六十一点八三

chartEntry(百分之六十一点八三, 0, 9, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 择人

chartEntry(择人, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 麦克拉伦

chartEntry(麦克拉伦, 0, 4, -14.0545195309, None) 

Input: 法正研究从波黑撤军计划

Matched: 超越

chartEntry(超越, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 开始

chartEntry(开始, 0, 2, -10.6760079076, No

chartEntry(上升, 0, 2, -11.8528856697, None) 

Input: 法正研究从波黑撤军计划

Matched: 传达

chartEntry(传达, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 上千

chartEntry(上千, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 列车

chartEntry(列车, 0, 2, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 千瓦

chartEntry(千瓦, 0, 2, -14.791485125, None) 

Input: 法正研究从波黑撤军计划

Matched: 传输

chartEntry(传输, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 交通

chartEntry(交通, 0, 2, -11.2065226243, None) 

Input: 法正研究从波黑撤军计划

Matched: 要点

chartEntry(要点, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 合作制

chartEntry(合作制, 0, 3, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: ２２０１．７８亿

chartEntry(２２０１．７８亿, 0, 8, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 在内

chartEntry(在内, 0, 2, -13.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 持续

chartEntry(持续, 0, 2, -12.0545195309, None) 

Input: 法正研究从波黑撤军计划

Matched: 法制

chartEntry(法制, 0, 2, -14.0545195309, None) 

Input:

chartEntry(取代, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 陈可冀

chartEntry(陈可冀, 0, 3, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 棉纺织

chartEntry(棉纺织, 0, 3, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 五百六十六

chartEntry(五百六十六, 0, 5, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 跳台

chartEntry(跳台, 0, 2, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 露出

chartEntry(露出, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 广告

chartEntry(广告, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 证交所

chartEntry(证交所, 0, 3, -13.2065226243, None) 

Input: 法正研究从波黑撤军计划

Matched: 雄厚

chartEntry(雄厚, 0, 2, -13.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 销

chartEntry(销, 0, 1, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 宪章

chartEntry(宪章, 0, 2, -13.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 蓬勃

chartEntry(蓬勃, 0, 2, -13.791485125, None) 

Input: 法正研究从波黑撤军计划

Matched: 将

chartEntry(将, 0, 1, -7.88860759192, None) 

Input: 法正研究从


Input: 法正研究从波黑撤军计划

Matched: 保持

chartEntry(保持, 0, 2, -10.6485271712, None) 

Input: 法正研究从波黑撤军计划

Matched: 发表

chartEntry(发表, 0, 2, -11.3320535064, None) 

Input: 法正研究从波黑撤军计划

Matched: 趋势

chartEntry(趋势, 0, 2, -12.9170160071, None) 

Input: 法正研究从波黑撤军计划

Matched: 系统化

chartEntry(系统化, 0, 3, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 广州

chartEntry(广州, 0, 2, -11.9170160071, None) 

Input: 法正研究从波黑撤军计划

Matched: 专业科

chartEntry(专业科, 0, 3, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 思想

chartEntry(思想, 0, 2, -12.9170160071, None) 

Input: 法正研究从波黑撤军计划

Matched: 应变

chartEntry(应变, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 百分之二十三点六

chartEntry(百分之二十三点六, 0, 8, -14.791485125, None) 

Input: 法正研究从波黑撤军计划

Matched: ７．５亿

chartEntry(７．５亿, 0, 4, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 二千万

chartEntry(二千万, 0, 3, -14.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 二点五亿

chartEntry(二点五亿, 0, 4, -14.791485125, None) 

Input: 法正研究从波黑撤军计划

Matched: 奇景

chartE

...


Matched: 科技型

chartEntry(科技型, 0, 3, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 量入为出

chartEntry(量入为出, 0, 4, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 健康

chartEntry(健康, 0, 2, -11.8528856697, None) 

Input: 法正研究从波黑撤军计划

Matched: 优化

chartEntry(优化, 0, 2, -12.2889847845, None) 

Input: 法正研究从波黑撤军计划

Matched: 交纳

chartEntry(交纳, 0, 2, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 问世

chartEntry(问世, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 理想

chartEntry(理想, 0, 2, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 残疾人

chartEntry(残疾人, 0, 3, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 硫酸镍

chartEntry(硫酸镍, 0, 3, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 蒂华纳

chartEntry(蒂华纳, 0, 3, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 道·琼斯

chartEntry(道·琼斯, 0, 4, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 深

chartEntry(深, 0, 1, -12.1285201123, None) 

Input: 法正研究从波黑撤军计划

Matched: 双向

chartEntry(双向, 0, 2, -16.3764476257,


chartEntry(据悉, 0, 2, -11.6215601236, None) 

Input: 法正研究从波黑撤军计划

Matched: 注目

chartEntry(注目, 0, 2, -14.791485125, None) 

Input: 法正研究从波黑撤军计划

Matched: 数万

chartEntry(数万, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 这

chartEntry(这, 0, 1, -8.05001813862, None) 

Input: 法正研究从波黑撤军计划

Matched: 彩叶

chartEntry(彩叶, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 无人不晓

chartEntry(无人不晓, 0, 4, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 警惕

chartEntry(警惕, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 大气层

chartEntry(大气层, 0, 3, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 韩

chartEntry(韩, 0, 1, -12.6760079076, None) 

Input: 法正研究从波黑撤军计划

Matched: 来往

chartEntry(来往, 0, 2, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 局长

chartEntry(局长, 0, 2, -14.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 九十三点六五万

chartEntry(九十三点六五万, 0, 7, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 道歉

chartEntry(道歉, 0, 2, -16.3764476257, None) 

Input: 

chartEntry(殉难, 0, 2, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 滞纳金

chartEntry(滞纳金, 0, 3, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 火

chartEntry(火, 0, 1, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 暂时

chartEntry(暂时, 0, 2, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 印度

chartEntry(印度, 0, 2, -13.791485125, None) 

Input: 法正研究从波黑撤军计划

Matched: 金融界

chartEntry(金融界, 0, 3, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 妥善

chartEntry(妥善, 0, 2, -13.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 八百

chartEntry(八百, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 席位

chartEntry(席位, 0, 2, -14.0545195309, None) 

Input: 法正研究从波黑撤军计划

Matched: 希夫香卡尔·梅农

chartEntry(希夫香卡尔·梅农, 0, 8, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 二百八十七

chartEntry(二百八十七, 0, 5, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 特大

chartEntry(特大, 0, 2, -14.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 世运

chartEntry(世运, 0, 2, -16.3764476257, None) 



Matched: 工商

chartEntry(工商, 0, 2, -13.2065226243, None) 

Input: 法正研究从波黑撤军计划

Matched: 二十七点八亿

chartEntry(二十七点八亿, 0, 6, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 地级

chartEntry(地级, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: ４月

chartEntry(４月, 0, 2, -12.791485125, None) 

Input: 法正研究从波黑撤军计划

Matched: 海岛

chartEntry(海岛, 0, 2, -14.791485125, None) 

Input: 法正研究从波黑撤军计划

Matched: 伙伴

chartEntry(伙伴, 0, 2, -12.2065226243, None) 

Input: 法正研究从波黑撤军计划

Matched: 海峡

chartEntry(海峡, 0, 2, -12.791485125, None) 

Input: 法正研究从波黑撤军计划

Matched: 人士

chartEntry(人士, 0, 2, -11.732591436, None) 

Input: 法正研究从波黑撤军计划

Matched: 七点八亿

chartEntry(七点八亿, 0, 4, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 入股

chartEntry(入股, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 促进

chartEntry(促进, 0, 2, -10.791485125, None) 

Input: 法正研究从波黑撤军计划

Matched: 单独

chartEntry(单独, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 提倡

chartEntry(提倡, 0, 2, -16.3764476257, None) 



Input: 法正研究从波黑撤军计划

Matched: 西宁市

chartEntry(西宁市, 0, 3, -14.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 土质

chartEntry(土质, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 结合部

chartEntry(结合部, 0, 3, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 证实

chartEntry(证实, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 风陵渡

chartEntry(风陵渡, 0, 3, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 洋溢

chartEntry(洋溢, 0, 2, -14.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 硅

chartEntry(硅, 0, 1, -15.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 集

chartEntry(集, 0, 1, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 又

chartEntry(又, 0, 1, -11.0188956211, None) 

Input: 法正研究从波黑撤军计划

Matched: 怀念

chartEntry(怀念, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 互相

chartEntry(互相, 0, 2, -14.791485125, None) 

Input: 法正研究从波黑撤军计划

Matched: 棉纺

chartEntry(棉纺, 0, 2, -16.3764476257, None) 

Input: 法正研究从波黑撤军计划

Matched: 行星

chartEntry(行星, 0, 2, -14.791485

In [ ]:
"""
Implimenting baseline algorithem.
Each line of the input file is one sentence. 
We initilize the heap by matching the count_1w and then use DP to find the best matching segmentation.
"""
def initHeap(input_filename='../data/input', counts_filename='../data/count_1w.txt', \
             sort_acc_to='log_prob'):
        